In [1]:
from collections import defaultdict
import json
import random
import secrets

In [2]:
# Simulation constants
CATEGORIES = ['eth_core', 'op_tooling', 'op_rnd']
NUM_CATEGORIES = len(CATEGORIES)
NUM_PROJECTS = 100
NUM_VOTERS = 100
MAX_OP = 1_000_000
MIN_OP = 1000
TOTAL_OP = 8_000_000
MAX_COI_PROJECTS = 10

In [3]:
# Initialize categories and projects

# Randomly generate project UIDs
projects = [
    f"0x{secrets.token_hex(32)}"
    for _ in range(NUM_PROJECTS)
]

# Randomly allocate projects to categories
projects_by_category = defaultdict(list)
for project in projects:
    category = random.choice(CATEGORIES)
    projects_by_category[category].append(project)

# Test
projects_by_category['op_tooling'][:5]

['0xfa636fa46131cb77a649998711fcf85ed9d03043f81a03601c58dafd67a1a7f4',
 '0xa8674c328a4993d6aa956985666d19eeb18599146dd6256af6390cb290fad145',
 '0x05a0e52bd1f8e36356edb50b0228a27d40744b7ab0e77f4db076043ec0982e1a',
 '0xfe01389f86caeb50e682acf04c6450bb2286156152f1c3730dc11c373b0b8956',
 '0x637e9769e9b494db77a683868d944668c2bdf40311ed2037aaa7c5ad5cf33867']

In [4]:
# Generate voter ballots
def generate_ballot():

    # Generate random category for the voter
    assigned_category = random.sample(CATEGORIES, 1)[0]

    # Generate random COI list for each voter (for simplicity, random projects)
    coi_projects = list(random.sample(projects, random.randint(0, MAX_COI_PROJECTS)))

    # Create a random threshold (up to 10%) for the voter to "zero" vote projects they don't like
    zero_thresh = random.randint(0, 1000)/100 
    
    #  Determine category allocations
    category_allocations = {category: random.randint(0, 10000)/100. for category in CATEGORIES}
    total = sum(category_allocations.values())
    category_allocations = {k: (v * 100 / total) for k, v in category_allocations.items()}

    # Determine project allocations within a category
    for category, proj_list in projects_by_category.items():
        if category != assigned_category:
            continue

        project_allocations = defaultdict(list)
        for project in proj_list:            
            alloc = random.randint(0, 10000)/100.
            if alloc < zero_thresh or project in coi_projects:
                alloc = 0
            project_allocations[project] = alloc
        
        total = sum(project_allocations.values())
        project_allocations = {
            k: v * 100 / total if k not in coi_projects else None
            for k, v in project_allocations.items()
        }

    return {
        "voter_id": f"0x{secrets.token_hex(20)}",
        "category_assignment": assigned_category,
        "category_allocations": category_allocations,
        "project_allocations": project_allocations
    }

# Generate all voter ballots
voter_ballots = [generate_ballot() for _ in range(NUM_VOTERS)]

In [5]:
voter_ballots[-3]

{'voter_id': '0xd981dccbd77a9d5c6a4cf56f282ea37f0dd3a03e',
 'category_assignment': 'op_tooling',
 'category_allocations': {'eth_core': 57.13966147192604,
  'op_tooling': 35.00857505033182,
  'op_rnd': 7.8517634777421526},
 'project_allocations': {'0xfa636fa46131cb77a649998711fcf85ed9d03043f81a03601c58dafd67a1a7f4': 5.137959129745443,
  '0xa8674c328a4993d6aa956985666d19eeb18599146dd6256af6390cb290fad145': 0.7074121377609444,
  '0x05a0e52bd1f8e36356edb50b0228a27d40744b7ab0e77f4db076043ec0982e1a': 4.050691447194575,
  '0xfe01389f86caeb50e682acf04c6450bb2286156152f1c3730dc11c373b0b8956': 5.096119968290321,
  '0x637e9769e9b494db77a683868d944668c2bdf40311ed2037aaa7c5ad5cf33867': 2.830199066326082,
  '0xb88e8372eeca84abe0912315160dfe7859627aec3d23a28f3d179e95628e5cc4': 4.208138817933587,
  '0x44242a24f31bb88fd8ed33e56e627640950ec038ee036860bb8d0920f8d47ace': 3.537060688804722,
  '0x53efdebc2838bd0be6653421be0d4ed63d4e9813bcf09b448cd8273eecf26df5': 4.065004844534485,
  '0xa3816b50347a001a0c24a

In [6]:
with open("rf5_synthetic_ballots.json", "w") as f:
    json.dump(voter_ballots, f, indent=2)